# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *"A non-local algorithm for image denoising"* (autorzy: Antoni Buades, Bartomeu Coll i Jean Michel Morel) na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{2}
\end{equation}

gdzie:
\begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{3}
\end{equation}

- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując: jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
import copy
import cv2
from scipy import signal
from numpy import linalg 
import time


if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate


In [ ]:
mat = loadmat('MR_data.mat')
Input_0 = mat['I_noisefree']
Input_1 = mat['I_noisy1']
Input_2 = mat['I_noisy2']
Input_3 = mat['I_noisy3']
Input_4 = mat['I_noisy4']

In [ ]:
def fgaussian(size, sigma):
    m = n = size
    h, k = m // 2, n // 2
    x, y = np.mgrid[-h:h+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g /g.sum() 

def mesh(size, sigma):
    m = n = size
    h, k = m//2, n//2
    x, y = np.mgrid[-h:h+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g 

In [ ]:
def wsp(okno, filtr, wariancja):
    A,B = okno.shape
    piksel = 0
    x = [A // 2, B // 2]
    for i in range(A):
        for j in range(B):
            AB = [i, j]
            y = np.sqrt(((x[0] - i)**2) + ((x[1] - j)**2))
            g = np.exp(-(y**2)/(2*(wariancja**2)))
            piksel = piksel + g * okno[i, j]
    piksel = piksel / filtr.sum()
    return piksel


def konwolucja(obraz, okno, wariancja):
    filtr = mesh(5, wariancja)
    IConv = obraz.copy()
    (X,Y) = IConv.shape
    polowa_okna = okno // 2
    for i in range(0 + polowa_okna, X - polowa_okna):
        for j in range(0 + polowa_okna, Y - polowa_okna):
            okno = IConv[i - polowa_okna : i + polowa_okna + 1, j - polowa_okna : j + polowa_okna + 1]
            new_pixel = wsp(okno, filtr, wariancja)
            IConv[i, j] = new_pixel
    return IConv


def exp(y, sigma):
  return np.exp(-(y*y/2*sigma*sigma))

def gamma(tab, sigma, Ix):
  tab_new = np.zeros((3, 3))
  for i in range(len(tab)):
    for j in range(len(tab[0])):
      tab_new[i, j] = exp(abs(tab[i, j] - Ix), sigma)
  return tab_new

def wsp_2(okno, filtr, wariancja, delta_r):
    A,B = okno.shape
    piksel_2 = 0
    normalizacja = 0
    
    tab = [A // 2, B // 2]
    for i in range(A):
        for j in range(B):
            AB = [i, j]
            
            y = np.sqrt(((tab[0] - i)**2) + ((tab[1] - j)**2))
            gauss = np.exp(-(y**2) / (2*(wariancja**2)))
            
            roznica = np.abs(okno[A // 2, B // 2] - okno[i, j])
            gauss_roznica = np.exp(-(roznica**2) / (2*(delta_r**2)))
            
            piksel_2 = piksel_2 + gauss * gauss_roznica * okno[i, j]
            normalizacja = normalizacja+(gauss * gauss_roznica)
    piksel_2 = piksel_2 / normalizacja
    return piksel_2
        
def bilateralna(obraz, okno, wariancja, delta_r):
    filtr = mesh(okno, wariancja)
    IConvol = obraz.copy()
    (X,Y) = IConvol.shape
    polowa_okna = okno//2
    for i in range(0 + polowa_okna, X - polowa_okna):
        for j in range(0 + polowa_okna, Y - polowa_okna):
            okno = IConvol[i - polowa_okna : i + polowa_okna + 1, j - polowa_okna : j + polowa_okna + 1]
            nowy_pixel = wsp_2(okno, filtr, wariancja, delta_r)
            IConvol[i,j] = nowy_pixel
    return IConvol

In [ ]:
#Implementacja filtra NML

def NLM(obraz, okno, obszar, alfa, sigma):
    
    #Zdefiniowanie obszaru poszukiwan piksela
    V = fgaussian(okno, np.sqrt(sigma))
    
    #Obraz wyjściwy
    Obrazwyj = obraz.copy()
    (X,Y) = Obrazwyj.shape
    polowa_obszaru = obszar//2
    polowa_okna = okno//2
    
    #Przejscie po pikselach oraz po obszarach przeszukiwan
    for i in range(polowa_okna + polowa_obszaru, X - polowa_obszaru - polowa_okna):
        for j in range(polowa_okna + polowa_obszaru, Y - polowa_obszaru - polowa_okna):
            
            i1 = i + polowa_obszaru + polowa_okna
            i2 = i - polowa_obszaru - polowa_okna
            j1 = j + polowa_obszaru + polowa_okna
            j2 = j - polowa_obszaru - polowa_okna
            
            #definicja obrazu wejsciowego
            nowy_obszar = obraz[i2 : i1, j2 : j1]
            A, B = nowy_obszar.shape
            
            #Definicja zmiennych
            zx = 0
            wpx = 0
            nowy_piksel = 0
            
            #wspolrzedne piksela
            x = [i, j]
            
            
            #podwojne petla dla kolejnych Np i wyliczenia wag
            for a in range(polowa_okna, A - polowa_okna):
                for b in range(polowa_okna, B - polowa_okna):
                    
                    #definicja otoczenia
                    Np = nowy_obszar[a-polowa_okna:a+polowa_okna+1,b-polowa_okna:b+polowa_okna+1]
                    
                    #definicja kontekstu
                    Nx = obraz[b-polowa_okna:b+polowa_okna+1,b-polowa_okna:b+polowa_okna+1]
                    
                    #Wymnożenie przez maskę Gaussa
                    V1 = V*Np
                    V2 = V*Nx
                    
                    #Odjęcie macierzy od siebie
                    Xk = V1 - V2
                    
                    #Kwadrat z normy
                    Xnorm = np.sqrt((Xk**2).sum())
                    
                    #Obliczenie wagi w(p,x)
                    wpx = np.exp((-(Xnorm**2))/(alfa * sigma))
                    
                    zx = zx + wpx
                    nowy_piksel = nowy_piksel + wpx * nowy_obszar[a, b]
            
            nowy_piksel = nowy_piksel / zx
            
            Obrazwyj[i, j] = nowy_piksel
            
    return Obrazwyj

In [ ]:
#Zdefiniowanie parametrow
okno = 6
wariancja = 0.7
delta_r = 33
wariancja1 = 4

#Obliczenie czasu dla konwulacji
start_conv = time.time()
konwolucja1 = konwolucja(Input_2, okno, wariancja)
stop_conv = time.time()
czas_conv = stop_conv - start_conv

#Oblicenie czasu dla filtracji bilateralnej
start_bilateralna = time.time()
bilateralna1 = bilateralna(Input_2, okno, wariancja1, delta_r)
stop_bilateralna = time.time()
czas_bilateralna = stop_bilateralna - start_bilateralna

#Obliczenie czasu dla filtracji NLM
start_NLM = time.time()
NLM1 = NLM(Input_2, 3, 8, 5, 45)
stop_NLM = time.time()
czas_NLM = stop_NLM - start_NLM

print("Porównanie czasów:")
print("Konwolucja - {}".format(czas_conv))
print("Filtracja bilateralna - {}".format(czas_bilateralna))
print("Filtracja NLM - {}".format(czas_NLM))

f, ax = plt.subplots(2, 2, figsize = (16, 16))

ax[0,0].imshow(Input_2, 'gray')
ax[0,0].set_title("Oryginalny obraz")
ax[0,0].axis('off')

ax[0,1].imshow(NLM1, 'gray')
ax[0,1].set_title("Filtracja NLM")
ax[0,1].axis('off')

ax[1,0].imshow(bilateralna1, 'gray')
ax[1,0].set_title("Filtracja bilateralna")
ax[1,0].axis('off')

ax[1,1].imshow(konwolucja1, 'gray')
ax[1,1].set_title("Konwolucja")
ax[1,1].axis('off')

plt.show()

**KOMENTARZ**   
Podsumowując na podstawie powyższych obrazów możemy stwierdzić, iż najlepszy wynik otrzymaliśmy dla Filtracji NML.Metoda ta zdecydowanie wygrywa z filtracją Gaussa oraz filtracją bilateralną pod tym względem, że dla filtacji NML został zlikwiodowany szum oraz krawędzie są widoczne. Czasy działania filtracji Gaussa oraz bilateralnej są bradzo podobne natomiast dla filtracji NML ten czas jest zdecydowanie dłuższy oraz metoda ta wykorzystuje zdecydowanie więcej pamięci.
